In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction


# Importing Packages

Declare the variables that will be in use throughout the notebook.

In [1]:
PACKAGE_PATH = 'Water_classification_package'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 4
drwxr-xr-x 1 root root 4096 Jul 13 13:43 sample_data
total 0
-rw-r--r-- 1 root root 0 Jul 24 11:27 __init__.py


In [2]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

# Acc = TP + TN / (TP + TN + FP + FN)
# possible_pos = TP + FN
# predicted_pos = TP + FP
# Missing TN
# TN = total - possible_pos - predicted_pos + TP
# TP + TN + FP + FN = possible_pos + predicted_pos - TP + TN

def custom_accuracy(y_true, y_pred):
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
  
    # total_data = tf.cast(total_data, tf.float32|tf.int32)
    # true_positives = tf.cast(true_positives, tf.float32|tf.int32)
    # possible_positives = tf.cast(possible_positives, tf.float32|tf.int32)
    # predicted_positives = tf.cast(predicted_positives, tf.float32|tf.int32)
    # print(K.int_shape(y_true), K.int_shape(y_pred))
    # print(K.int_shape(y_pred)[0], K.int_shape(y_pred)[1], K.int_shape(y_pred)[2])
    # print(total_data)
    # print(possible_positives)
    # (true_positives) / (total_data + K.epsilon())
    return (true_positives + true_negatives) / (total_data + K.epsilon())



def MetricCalculator(model, test_data, total_steps):
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  # total_steps = 2000
  test_acc_metric = tf.keras.metrics.Accuracy()
  # test_F1_metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5)
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    # print(y_pred[0][1] == 1, y_pred[0][1] == 1)
    for j in range(y_pred.shape[0]):
      if y_true[j][1] == 1 and y_pred[j][1] == 1:
        TP += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 0:
        TN += 1
      if y_true[j][1] == 1 and y_pred[j][1] == 0:
        FN += 1
      if y_true[j][1] == 0 and y_pred[j][1] == 1:
        FP += 1
    test_acc_metric.update_state(y_true, y_pred)
    # # recall_metric.update_state(data[1], test_logits)
    # # precision_metric.update_state(data[1], test_logits)
  print("TP: ", TP)
  print("TN: ", TN)
  print("FP: ", FP)
  print("FN: ", FN)
  if TP != 0:
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    F1 = 2*(recall*precision)/(recall + precision)
  else:
    precision = None
    recall = None
    F1 = None

  print("precision: ", precision)
  print("recall: ", recall)
  print("F1_Score: : ", F1)
  print("Accuracy: ", test_acc_metric.result().numpy())
  return F1, test_acc_metric.result().numpy()

Writing Water_classification_package/metrics_.py


In [3]:
%%writefile {PACKAGE_PATH}/config.py
import tensorflow as tf
from . import metrics_
class configuration:
  def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE, BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None, type_=1):
    if type_ == 1:
      self.type_ = "fs"
    elif type_ == 2:
      self.type_ = "m2"
    elif type_ == 3:
      self.type_ = "m3"
    else:
      self.type_ = None
    self.country = country
    self.PROJECT_TITLE = PROJECT_TITLE
    self.BANDS1 = BANDS1
    self.BANDS2 = BANDS2
    self.BANDS3 = BANDS3
    self.BUCKET = "geebucketwater"
    self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
    self.TRAIN_SIZE = TRAIN_SIZE
    self.EVAL_SIZE = EVAL_SIZE
    self.BUCKET = "geebucketwater"
    self.TRAINING_BASE = f'training_patches'
    self.EVAL_BASE = f'eval_patches'
    self.TEST_BASE_1 = f'test_patches_1'
    self.TEST_BASE_2 = f'test_patches_2'
    self.TEST_BASE_3 = f'test_patches_3'
    self.TEST_BASE_4 = f'test_patches_4'
    self.TEST_BASE_5 = f'test_patches_5'
    self.TEST_BASE_6 = f'test_patches_6'
    self.TEST_BASE_7 = f'test_patches_7'
    self.TEST_BASE_8 = f'test_patches_8'
    self.TEST_BASE_9 = f'test_patches_9'
    self.TEST_BASE_10 = f'test_patches_10'
    self.TEST_BASE = f'test_patches'
    self.RESPONSE = 'water'
    self.BANDS = BANDS1 + BANDS2 + BANDS3 
    self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
    # Specify the size and shape of patches expected by the model.
    self.KERNEL_SIZE = 256
    self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
    self.COLUMNS = [
      tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32) for k in self.FEATURES
    ]
    self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
    # Specify model training parameters.
    self.BATCH_SIZE = 16
    self.EPOCHS = 5
    self.BUFFER_SIZE = 2000
    self.OPTIMIZER = 'adam'
    self.LOSS = 'categorical_crossentropy'
    self.METRICS = ['AUC', metrics_.f1, metrics_.custom_accuracy]
    self.image = image
    self.sam_arr = sam_arr
    




Writing Water_classification_package/config.py


In [4]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf

class Preprocessor:
  def __init__(self, config):
    self.config = config

  def parse_tfrecord(self, example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
      example_proto: a serialized Example.
    Returns:
      A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, self.config.FEATURES_DICT)


  def to_tuple(self, inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
      inputs: A dictionary of tensors, keyed by feature name.
    Returns:
      A tuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in self.config.FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(self.config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(self.config.BANDS):], tf.int32), depth=2),[256,256,2])


  def get_dataset(self, pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
      pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
      A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
    return dataset

  def get_training_dataset(self):
    """Get the preprocessed training dataset
    Returns: 
      A tf.data.Dataset of training data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + self.config.FOLDER + '/' + self.config.TRAINING_BASE + '*'
    print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.shuffle(self.config.BUFFER_SIZE).batch(self.config.BATCH_SIZE).repeat()
    return dataset



  def get_eval_dataset(self):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + self.config.FOLDER + '/' + self.config.EVAL_BASE + '*'
    print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset

  # print(iter(evaluation.take(1)).next())

  def get_test_dataset(self, test_base):
    """Get the preprocessed evaluation dataset
    Returns: 
      A tf.data.Dataset of evaluation data.
    """
    glob = 'gs://' + self.config.BUCKET + '/' + self.config.FOLDER + '/' + test_base + '*'
    print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.batch(1).repeat()
    return dataset



Writing Water_classification_package/preprocessing.py


In [5]:
import json
from pprint import pprint
import numpy as np
import tqdm.notebook as tq


def doExport(out_image_base, kernel_buffer, region, setting):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = setting.image.select(setting.BANDS),
    description = out_image_base,
    bucket = setting.BUCKET,
    fileNamePrefix = setting.FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': setting.KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

def LoadImage(out_image_base, user_folder, kernel_buffer, setting):
  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{setting.BUCKET}'/'{setting.FOLDER} # noqa

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  pprint(imageFilesList)
  print(jsonFile)

  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile} # noqa
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      setting.KERNEL_SHAPE[0] + kernel_buffer[0],
      setting.KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in setting.BANDS
  ]

  imageFeaturesDict = dict(zip(setting.BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in setting.BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  return imageDataset, patches, x_buffer, y_buffer, jsonFile

def predictionSingleinput(model, imageDataset, patches):
  print('Running predictions...')
  predictions = model.predict(imageDataset, steps=patches, verbose=1)
  return predictions

def predictionMultipleinput(model, imageDataset, patches, setting):
  print('Running predictions...')
  predictions = []
  pbar = tq.tqdm(total=patches)
  for data in imageDataset:
    pbar.update(1)
    x1, x2 = tf.split(data, [len(setting.BANDS1), len(setting.BANDS2)], 3)
    predictions.append(model_custom.predict([x1, x2], verbose=0))
  return predictions

def predictionMultipleinput_3(model, imageDataset, patches, setting):
  print('Running predictions...')
  predictions = []
  pbar = tq.tqdm(total=patches)
  for data in imageDataset:
    pbar.update(1)
    x1, x2, x3 = tf.split(data, [len(setting.BANDS1), len(setting.BANDS2), len(setting.BANDS3)], 3)
    predictions.append(model_custom.predict([x1, x2, x3], verbose=0))
  return predictions

def uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, multiview=False):
  print('Writing predictions...')
  out_image_file = 'gs://' + setting.BUCKET + '/' + setting.FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    if multiview == True:
      predictionPatch = predictionPatch[0]
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+setting.KERNEL_SIZE, y_buffer:y_buffer+setting.KERNEL_SIZE]
    predictionPatch = np.argmax(predictionPatch, -1)
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base + suffix
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

def doPrediction_featurestack(out_image_base, user_folder, kernel_buffer, model, suffix, setting):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  imageDataset, patches, x_buffer, y_buffer, jsonFile = LoadImage(out_image_base, user_folder, kernel_buffer, setting)
  predictions = predictionSingleinput(model, imageDataset, patches)
  uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, False)
  return

def doPrediction_multiview_2(out_image_base, user_folder, kernel_buffer, model, suffix, setting):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  imageDataset, patches, x_buffer, y_buffer, jsonFile = LoadImage(out_image_base, user_folder, kernel_buffer, setting)
  predictions = predictionMultipleinput(model, imageDataset, patches, setting)
  uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, True)
  return

def doPrediction_multiview_3(out_image_base, user_folder, kernel_buffer, model, suffix, setting):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  imageDataset, patches, x_buffer, y_buffer, jsonFile = LoadImage(out_image_base, user_folder, kernel_buffer, setting)
  predictions = predictionMultipleinput_3(model, imageDataset, patches, setting)
  uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, True)
  return

In [6]:
%%writefile {PACKAGE_PATH}/sampling.py
import ee
# Convert the feature collections to lists for iteration.
# trainingPolysList = trainingPolys.toList(trainingPolys.size())
# evalPolysList = evalPolys.toList(evalPolys.size())

def Training_task(trainingPolys, n, N, arrays, setting, foldername):
  trainingPolysList = trainingPolys.toList(trainingPolys.size())
  # Export all the training data (in many pieces), ith one task 
  # per geometry.
  for g in range(trainingPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(trainingPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n, # Size of the shard.
        seed = i,
        tileScale = 8
      )
      geomSample = geomSample.merge(sample)
    
    desc = setting.TRAINING_BASE + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()


def Eval_task(evalPolys, n, N, arrays, setting, foldername):
  evalPolysList = evalPolys.toList(evalPolys.size())
  # Export all the evaluation data.
  for g in range(evalPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(evalPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n,
        seed = i,
        tileScale = 8
      )
      # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
      geomSample = geomSample.merge(sample)

    desc = setting.EVAL_BASE + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()

# testPolys1List = testPolys1.toList(testPolys1.size())
# testPolys2List = testPolys2.toList(testPolys2.size())

def Testing_task(testPolys, n, N, arrays, setting, foldername, Test_base):
  # Export all the test data.
  testPolysList = testPolys.toList(testPolys.size())
  for g in range(testPolys.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(testPolysList.get(g)).geometry(), 
        scale = 30,
        numPixels = N / n,
        seed = i,
        tileScale = 8
      )
      # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
      geomSample = geomSample.merge(sample)

    desc = Test_base + '_g' + str(g)
    task = ee.batch.Export.table.toCloudStorage(
      collection = geomSample,
      description = desc,
      bucket = setting.BUCKET,
      fileNamePrefix = foldername + '/' + desc,
      fileFormat = 'TFRecord',
      selectors = setting.BANDS + [setting.RESPONSE]
    )
    task.start()



Writing Water_classification_package/sampling.py


In [7]:
# def Testing_task2(testPolys2, n, N, arrays, setting, foldername):
#   testPolys2List = testPolys2.toList(testPolys2.size())
#   # Export all the test2 data.
#   for g in range(testPolys2.size().getInfo()):
#     geomSample = ee.FeatureCollection([])
#     for i in range(n):
#       sample = arrays.sample(
#         region = ee.Feature(testPolys2List.get(g)).geometry(), 
#         scale = 30,
#         numPixels = N / n,
#         seed = i,
#         tileScale = 8
#       )
#       # geomSample = geomSample.filter(ee.Filter.greaterThan('water', -1))
#       geomSample = geomSample.merge(sample)

#     desc = setting.TEST_BASE_2 + '_g' + str(g)
#     task = ee.batch.Export.table.toCloudStorage(
#       collection = geomSample,
#       description = desc,
#       bucket = setting.BUCKET,
#       fileNamePrefix = foldername + '/' + desc,
#       fileFormat = 'TFRecord',
#       selectors = setting.BANDS + [setting.RESPONSE]
#     )
#     task.start()

# Setup software libraries

Authenticate and import as necessary.

In [8]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MssQilUc1sqkFO_y2odLErheP9yAMKlMLCAPR1GXyIw&tc=zVQudpVZOnFhZywgCq8-Z_-AxSNrVduK9gENxaRw7-U&cc=qDSoZWXwAO0Qe9pdjefTeY7GxaoH7SxKoO6ecOsvoRU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjbXdPwKKdJtqmmG_oIwLLj_V0VH_uzQaFUw7V6uG8m52BQf8Z_4Sc

Successfully saved authorization token.
Updated property [core/project].
2.8.2


In [9]:
from importlib import reload
# reload(config)
# reload(sampling)
from Water_classification_package import preprocessing, config, metrics_, sampling

# Imagery

Load data

In [10]:
# Use Landsat 8 surface reflectance data.
S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filterDate('2018-01-01','2018-02-01') \

S1A = S1.median()
S1 = S1.select('VV', 'VH').median()

elevation = ee.Image('NASA/NASADEM_HGT/001').select('elevation');

slope = ee.Terrain.slope(elevation);

aspect = ee.Terrain.aspect(elevation);

# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-01-01','2018-02-01')

# Cloud masking function.
def maskL8sr(image):
  BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

L8SR = l8sr.map(maskL8sr).median()


waterdata = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filterDate('2018-01-01', '2018-02-01').median()
watermask = waterdata.select("water")
mask = watermask.gt(0)
maskedComposite = waterdata.updateMask(mask).subtract(1)

# Setting up Config

Stack the 2D images (Landsat composite and NLCD impervious surface) to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

## Training Globally

In [11]:
configs_fs_global = {}
train_size = 72*10
eval_size = 72*3

#### Feature stack experiment

configs_fs_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, country = "global")
configs_fs_global["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size, country = "global")

###### Multiexperiment

configs_multi_global = {}

configs_multi_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")


configs_multi_global["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2, country = "global")
configs_multi_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2, country = "global")

configs_multi_3_global = {}

configs_multi_3_global["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3, country = "global")

configs_multi_3_global["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

In [12]:
print(configs_multi_global["L8SR_S1A"].FOLDER)
print(configs_multi_global["L8SR_S1A"].BANDS1)
print(configs_multi_global["L8SR_S1A"].BANDS2)
print(configs_multi_global["L8SR_S1A"].BANDS)
print(configs_multi_global["L8SR_S1A"].TRAINING_BASE)
print(configs_multi_global["L8SR_S1A"].EVAL_BASE)
print(configs_multi_global["L8SR_S1A"].TEST_BASE_1)
print(configs_multi_global["L8SR_S1A"].TEST_BASE_2)
print(configs_multi_global["L8SR_S1A"].country)
print(configs_multi_global["L8SR_S1A"].type_)

m2_global_Cnn_L8SR_S1A
['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
['VV', 'VH', 'angle']
['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'VV', 'VH', 'angle']
training_patches
eval_patches
test_patches_1
test_patches_2
global
m2


In [13]:
# Feature Stack

configs_fs_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float()
configs_fs_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float()
configs_fs_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float()
configs_fs_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float()
configs_fs_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float()
configs_fs_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float()
configs_fs_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float()

configs_fs_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()
configs_fs_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_fs_global["S1_sl"].image = ee.Image.cat([S1, slope]).float()
configs_fs_global["S1_as"].image = ee.Image.cat([S1, aspect]).float()
configs_fs_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_fs_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float()
configs_fs_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float()

configs_fs_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float()
configs_fs_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float()
configs_fs_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float()
configs_fs_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float()
configs_fs_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float()
configs_fs_global["L8SR_el_sl"].image = ee.Image.cat([L8SR, elevation, slope]).float()
configs_fs_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float()

configs_fs_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float()
configs_fs_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float()
configs_fs_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float()

configs_fs_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float()
configs_fs_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float()
configs_fs_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float()

configs_fs_global["L8SR"].image = L8SR.float()
configs_fs_global["S1"].image = S1.float()
configs_fs_global["S1A"].image = S1A.float()
configs_fs_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float()
configs_fs_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float()


# Multiview

configs_multi_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float() 
configs_multi_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float() 
configs_multi_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float() 
configs_multi_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float() 
configs_multi_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float() 
configs_multi_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float() 
configs_multi_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float() 

configs_multi_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()  
configs_multi_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_multi_global["S1_sl"].image = ee.Image.cat([S1, slope]).float() 
configs_multi_global["S1_as"].image = ee.Image.cat([S1, aspect]).float() 
configs_multi_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_multi_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float() 
configs_multi_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float() 

configs_multi_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float() 
configs_multi_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float() 
configs_multi_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float() 
configs_multi_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float() 
configs_multi_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float() 
configs_multi_global["L8SR_el_sl"].image = ee.Image.cat([ L8SR, elevation, slope]).float() 
configs_multi_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float() 


configs_multi_global["L8SR_S1_as"].image = ee.Image.cat([L8SR, S1, aspect]).float()
configs_multi_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float() 
configs_multi_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float() 

configs_multi_global["L8SR_S1_sl_as"].image = ee.Image.cat([L8SR, S1, slope, aspect]).float() 
configs_multi_global["L8SR_S1_el_sl"].image = ee.Image.cat([L8SR, S1, elevation, slope]).float() 
configs_multi_global["L8SR_S1_el_as"].image = ee.Image.cat([L8SR, slope, elevation, aspect]).float() 
configs_multi_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1A_as"].image = ee.Image.cat([L8SR, S1A, aspect]).float()
configs_multi_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float() 
configs_multi_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float() 

configs_multi_global["L8SR_S1A_sl_as"].image = ee.Image.cat([L8SR, S1A, slope, aspect]).float() 
configs_multi_global["L8SR_S1A_el_sl"].image = ee.Image.cat([L8SR, S1A, elevation, slope]).float() 
configs_multi_global["L8SR_S1A_el_as"].image = ee.Image.cat([L8SR, S1A, elevation, aspect]).float() 
configs_multi_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float() 
configs_multi_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float() 

# Multiview-3


configs_multi_3_global["L8SR_S1_as3"].image = ee.Image.cat([L8SR, S1, aspect])
configs_multi_3_global["L8SR_S1_el3"].image = ee.Image.cat([L8SR, S1, elevation])
configs_multi_3_global["L8SR_S1_sl3"].image = ee.Image.cat([L8SR, S1, slope])
configs_multi_3_global["L8SR_S1_sl_el_as3"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect])

configs_multi_3_global["L8SR_S1A_as3"].image = ee.Image.cat([L8SR, S1A, aspect])
configs_multi_3_global["L8SR_S1A_el3"].image = ee.Image.cat([L8SR, S1A, elevation])
configs_multi_3_global["L8SR_S1A_sl3"].image = ee.Image.cat([L8SR, S1A, slope])
configs_multi_3_global["L8SR_S1A_sl_el_as3"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect])


In [14]:
for key in list(configs_fs_global):
  settings = configs_fs_global[key]
  featureStack = ee.Image.cat([
  settings.image.select(settings.BANDS),
  maskedComposite.select(settings.RESPONSE)
  ]).float()
  list_ = ee.List.repeat(1, settings.KERNEL_SIZE)
  lists = ee.List.repeat(list_, settings.KERNEL_SIZE)
  kernel = ee.Kernel.fixed(settings.KERNEL_SIZE, settings.KERNEL_SIZE, lists)
  arrays = featureStack.neighborhoodToArray(kernel)
  configs_fs_global[key].sam_arr = arrays
  print(key, settings.sam_arr.getInfo())
  

S1A_el_sl_as {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'dimensions': [1288801, 421201], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'slope', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 2}, 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -179.0001388888889, 0, -0.0002777777777777778, 61.00013888888889]}, {'id': 'aspect', 'data_type': {'typ

# Sampling

## Sampling globally

Use some pre-made geometries to sample the stack in strategic locations.  Specifically, these are hand-made polygons in which to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation.



The mapped data look reasonable so take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [ ]:
import folium

India = ee.Geometry.BBox(84.28766453687791, 24.192675258975537, 87.49567234937791, 26.180892485228437)
Pakistan = ee.Geometry.BBox(70.3009325419945, 31.3604931755229, 71.5533739482445, 32.33570151710531)
Cambodia = ee.Geometry.BBox(105.35157579365571, 12.08228250180923, 106.32935899678071, 13.025952639601176)
Brazil = ee.Geometry.BBox(-60.060403802711626, -19.002700742809683, -53.204935052711626, -13.90505097174063)
Bangladesh = ee.Geometry.BBox(89.50323560069529, 23.090980419149876, 91.00286938975779, 24.077692617901942)
Mexico = ee.Geometry.BBox(-104.87430997262705, 19.986643103978352, -98.01884122262705, 26.32579195587412)
Egypt = ee.Geometry.BBox(30.120884854535284, 17.6368163032104, 34.169346768597784, 19.754417837576597)
Thailand = ee.Geometry.BBox(100.47883808661508, 5.9400179647667635, 101.59479267596963, 6.8385138873107945)
Ghana = ee.Geometry.BBox(-1.0995263958226564, 6.221477061378518, 0.44954586980234357, 7.639227255737275)
Tibet = ee.Geometry.BBox(88.13943211306636, 28.727971943391157, 91.17165867556636, 31.04238968021801)


trainingPolys_global = ee.FeatureCollection(India).merge(Pakistan).merge(Cambodia).merge(Brazil) \
                          .merge(Bangladesh).merge(Mexico).merge(Egypt).merge(Thailand).merge(Ghana) \
                          .merge(Tibet)

India_eval = ee.Geometry.BBox(75.68466076783874, 21.263314014804795, 78.04672131471374, 22.770654917923526)
Tibet_eval = ee.Geometry.BBox(93.92188430197075, 26.847874650118836, 95.93238234884575, 28.356873608833094)
brazil_eval = ee.Geometry.BBox(-53.749924908880686, -22.833543783979856, -51.047288190130686, -20.999024800255494)

evalPolys_global = ee.FeatureCollection(Tibet_eval).merge(India_eval).merge(brazil_eval)

thai_test = ee.Geometry.BBox(100.30632852425321, 17.709225431372587, 100.74128946175321, 18.20417872756825)
tibet_test = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)
ghana_test = ee.Geometry.BBox(-1.9983132238272127, 5.925449378444892, -1.8517086095694002, 6.081004042776062)
brazil_test = ee.Geometry.BBox(-63.02141262744682, -28.962951804200575, -55.59465481494682, -21.415208673846603)
mexico_test = ee.Geometry.BBox(-93.47672355917602, 15.8775670491606535, -90.70816887167602, 18.27183442641013)
pakistan_test = ee.Geometry.BBox(68.55305064706133, 27.98976293885938, 70.93708385018633, 29.42545415991563)
egypt_test = ee.Geometry.BBox(24.8505863665, 20.337905952546933, 33.1123051165, 26.856885967831754)
cambodia_test = ee.Geometry.BBox(102.74910463260488, 11.950119301574833, 104.88045228885488, 13.600069066335008)
India_test = ee.Geometry.BBox(80.42186850663656, 22.601971814608234, 83.05858725663656, 24.635163539106795)
Bangladesh_test = ee.Geometry.BBox(89.25478625914253, 24.205326578074743, 90.15566516539253, 24.765228419516816)

testPolys_global = ee.FeatureCollection(thai_test).merge(tibet_test).merge(ghana_test).merge(brazil_test) \
                          .merge(mexico_test).merge(pakistan_test).merge(egypt_test).merge(cambodia_test).merge(India_test) \
                          .merge(Bangladesh_test)

polyImage = ee.Image(0).byte().paint(trainingPolys_global, 1).paint(evalPolys_global, 2).paint(testPolys_global, 3)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 3, 'palette': ['red', 'blue', "green"]})
map = folium.Map(location=[16.426385350573945, 102.16833965985882], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
n = 6 # Number of shards in each polygon.
N = 72 # Total sample size in each polygon.
settings = configs_fs_global["L8SR_S1A_sl_el_as"] # This contains all the features
# imageExport
sampling.Training_task(trainingPolys_global, n, N, settings.sam_arr, settings, "train_in_global")
sampling.Eval_task(evalPolys_global, n, N, settings.sam_arr, settings, "train_in_global")
sampling.Testing_task(testPolys_global, n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE)

# sampling.Testing_task(ee.FeatureCollection(thai_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_1)
# sampling.Testing_task(ee.FeatureCollection(tibet_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_2)
# sampling.Testing_task(ee.FeatureCollection(ghana_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_3)
# sampling.Testing_task(ee.FeatureCollection(brazil_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_4)
# sampling.Testing_task(ee.FeatureCollection(mexico_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_5)
# sampling.Testing_task(ee.FeatureCollection(pakistan_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_6)
# sampling.Testing_task(ee.FeatureCollection(egypt_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_7)
# sampling.Testing_task(ee.FeatureCollection(cambodia_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_8)
# sampling.Testing_task(ee.FeatureCollection(India_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_9)
# sampling.Testing_task(ee.FeatureCollection(Bangladesh_test), n, N, settings.sam_arr, settings, "train_in_global", settings.TEST_BASE_10)

In [ ]:
# Print all tasks.
from pprint import pprint
pprint(ee.batch.Task.list())
# for i in range(2):
  # pprint(ee.batch.Task.list()[i])

[<Task FIIH73ZS6BI3MEBXRZ5647DC EXPORT_FEATURES: test_patches_10_g0 (READY)>,
 <Task C3VQNIYIKLBPQBGLUZVFOEUH EXPORT_FEATURES: test_patches_9_g0 (READY)>,
 <Task 42KRF6VYZA35GBSQCMUZRGSF EXPORT_FEATURES: test_patches_8_g0 (READY)>,
 <Task VENFYICWM7UFS7KKI33G2H2B EXPORT_FEATURES: test_patches_7_g0 (READY)>,
 <Task 2EJNNQMRJHKLGFK2X4FAKGAK EXPORT_FEATURES: test_patches_6_g0 (READY)>,
 <Task VVMOOOOZ2IVTC37FT6RZCZIA EXPORT_FEATURES: test_patches_5_g0 (READY)>,
 <Task NXJV4RABS6HEKV7XKSLCNEKY EXPORT_FEATURES: test_patches_4_g0 (READY)>,
 <Task SCS33OQYNSX6G6YDLCSA4REA EXPORT_FEATURES: test_patches_3_g0 (READY)>,
 <Task EEHHE3LOQ3EMTHIR5RI3POPU EXPORT_FEATURES: test_patches_2_g0 (READY)>,
 <Task BZXARW3CXQGDSZHCDWABZOGS EXPORT_FEATURES: test_patches_1_g0 (READY)>,
 <Task FOGJWVHHIND2ZWDTICRRIGM2 EXPORT_FEATURES: test_patches_g9 (READY)>,
 <Task 35OCRHFH6SV5TEUXZBMOEAK3 EXPORT_FEATURES: test_patches_g8 (READY)>,
 <Task EJTYLMLWYIZXIPMVUN6BUVH6 EXPORT_FEATURES: test_patches_g7 (READY)>,
 <Ta

In [ ]:
# !earthengine task cancel all

Canceling task "L5ZJN2IZ6YKWXLD5Q6HUEHOJ"
Canceling task "CKQZODRNXSSOLMPQAMGLR7TW"
Canceling task "KL2B7QTZSZHUDEE73HROSIGS"
Canceling task "43HMONZIVDVGYVQX32LIUSV2"
Canceling task "ISZBI4CHWZGG22O5JNIEHJCZ"
Canceling task "7IOL6STFSKJXBKPAXFTFDEZY"
Canceling task "4NOWTSSAXGPOTN7AA4N2WIDS"
Canceling task "TSD4VGQ5TQ2KL7WBRA2BVWAA"
Canceling task "3VNLRTAVH47YAE3M652SD2SN"
Canceling task "WZRXSTM76TEIVYONKINE4RSW"
Canceling task "6JRDYC4DTPPOZGD4UXY7MDWT"
Canceling task "PWDQER5WCU3YOBVCFVI52TRK"


# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
for _, key in enumerate(list(configs_fs)):
  conf = configs_fs[key]
  preproc = preprocessing.Preprocessor(conf)
  training = preproc.get_training_dataset()
  evaluation = preproc.get_eval_dataset()
  test_1 = preproc.get_test_dataset(config.TEST_BASE_1)
  test_2 = preproc.get_test_dataset(config.TEST_BASE_2)

gs://geebucketwater/fs_TH_Cnn_S1A_el_sl_as/training_patches_S1A_el_sl_as*
gs://geebucketwater/fs_TH_Cnn_S1A_el/training_patches_S1A_el*
gs://geebucketwater/fs_TH_Cnn_S1A_sl/training_patches_S1A_sl*
gs://geebucketwater/fs_TH_Cnn_S1A_as/training_patches_S1A_as*
gs://geebucketwater/fs_TH_Cnn_S1A_sl_as/training_patches_S1A_sl_as*
gs://geebucketwater/fs_TH_Cnn_S1A_el_sl/training_patches_S1A_el_sl*
gs://geebucketwater/fs_TH_Cnn_S1A_el_as/training_patches_S1A_el_as*
gs://geebucketwater/fs_TH_Cnn_S1_el_sl_as/training_patches_S1_el_sl_as*
gs://geebucketwater/fs_TH_Cnn_S1_el/training_patches_S1_el*
gs://geebucketwater/fs_TH_Cnn_S1_sl/training_patches_S1_sl*
gs://geebucketwater/fs_TH_Cnn_S1_as/training_patches_S1_as*
gs://geebucketwater/fs_TH_Cnn_S1_sl_as/training_patches_S1_sl_as*
gs://geebucketwater/fs_TH_Cnn_S1_el_sl/training_patches_S1_el_sl*
gs://geebucketwater/fs_TH_Cnn_S1_el_as/training_patches_S1_el_as*
gs://geebucketwater/fs_TH_Cnn_L8SR_el_sl_as/training_patches_L8SR_el_sl_as*
gs://geebu

# Export Images

In [16]:
conf = configs_multi_global["L8SR_S1A_sl"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})

# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]

th_image_base = f'Thai_with_Multi_2{conf.PROJECT_TITLE}'
th_region = ee.Geometry.BBox(100.30632852425321, 17.709225431372587, 100.74128946175321, 18.20417872756825)

tb_image_base = f'Tibet_with_Multi_2{conf.PROJECT_TITLE}'
tb_region = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)

ghana_image_base = f'Ghana_with_Multi_2{conf.PROJECT_TITLE}'
ghana_region = ee.Geometry.BBox(-1.9983132238272127, 5.925449378444892, -1.8517086095694002, 6.081004042776062)


# Export Image to be predicted

In [ ]:
# Run the export.
doExport(th_image_base, kernel_buffer, th_region, conf)

Running image export to Cloud Storage...
Image export completed.


In [ ]:
doExport(tb_image_base, kernel_buffer, tb_region, conf)

Running image export to Cloud Storage...
Image export completed.


In [17]:
doExport(ghana_image_base, kernel_buffer, ghana_region, conf)

Running image export to Cloud Storage...
Image export completed.


# Predict Feature stack

In [ ]:
doPrediction_featurestack(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_as00000.tfrecord.gz',
 'gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_as00001.tfrecord.gz']
gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_asmixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 6,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            100.30615411937102,
                                            0.0,
                                            -0.00026949458523585647,
                                            18.204359232682105]},
                'crs': 'EPSG:4326'},
 'totalPatches': 42}
Running predictions...
42/42 [==============================] - 25s 257ms/step
Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Wr

In [ ]:
doPrediction_featurestack(tb_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00000.tfrecord.gz',
 'gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00001.tfrecord.gz',
 'gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00002.tfrecord.gz',
 'gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00003.tfrecord.gz']
gs://geebucketwater/fs_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_asmixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            83.78640553899825,
                                            0.0,
                                            -0.00026949458523585647,
                                            31.624111599086813]},
                'crs': 'EPSG:4326'},
 'totalPatches': 80}
Running predictions...
80/80 [==============================] - 30s 354ms/step
Writin

# Predict Multiview-2

In [ ]:
doPrediction_multiview_2(th_image_base, user_folder, kernel_buffer, model_custom, "multiview-2_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_as00000.tfrecord.gz',
 'gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_as00001.tfrecord.gz']
gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Thai_with_L8SR_sl_asmixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 6,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            100.30615411937102,
                                            0.0,
                                            -0.00026949458523585647,
                                            18.204359232682105]},
                'crs': 'EPSG:4326'},
 'totalPatches': 42}
Running predictions...


  0%|          | 0/42 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13...
Writing patch 14...
Writing patch 15...
Writing patch 16...
Writing patch 17...
Writing patch 18...
Writing patch 19...
Writing patch 20...
Writing patch 21...
Writing patch 22...
Writing patch 23...
Writing patch 24...
Writing patch 25...
Writing patch 26...
Writing patch 27...
Writing patch 28...
Writing patch 29...
Writing patch 30...
Writing patch 31...
Writing patch 32...
Writing patch 33...
Writing patch 34...
Writing patch 35...
Writing patch 36...
Writing patch 37...
Writing patch 38...
Writing patch 39...
Writing patch 40...
Writing patch 41...
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a sup

[array([[[[0.40169165, 0.5983083 ],
          [0.3716907 , 0.6283093 ],
          [0.29202718, 0.70797276],
          ...,
          [0.35341832, 0.64658165],
          [0.37056774, 0.6294323 ],
          [0.4668972 , 0.5331028 ]],
 
         [[0.27519113, 0.7248089 ],
          [0.19806272, 0.8019373 ],
          [0.15469912, 0.8453009 ],
          ...,
          [0.17201042, 0.8279895 ],
          [0.26559284, 0.7344072 ],
          [0.3960571 , 0.6039429 ]],
 
         [[0.35866016, 0.64133984],
          [0.27727434, 0.72272563],
          [0.16433586, 0.83566415],
          ...,
          [0.17095612, 0.82904387],
          [0.25021738, 0.74978256],
          [0.4134791 , 0.58652097]],
 
         ...,
 
         [[0.39731148, 0.6026885 ],
          [0.26178756, 0.73821247],
          [0.23028131, 0.76971865],
          ...,
          [0.9967704 , 0.00322969],
          [0.9957033 , 0.00429676],
          [0.8820791 , 0.11792086]],
 
         [[0.37775293, 0.6222471 ],
          [0

In [ ]:
doPrediction_multiview_2(tb_image_base, user_folder, kernel_buffer, model_custom, "multiview-2_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00000.tfrecord.gz',
 'gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00001.tfrecord.gz',
 'gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00002.tfrecord.gz',
 'gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_as00003.tfrecord.gz']
gs://geebucketwater/m2_TH_Cnn_L8SR_sl_as/Tibet_with_L8SR_sl_asmixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            83.78640553899825,
                                            0.0,
                                            -0.00026949458523585647,
                                            31.624111599086813]},
                'crs': 'EPSG:4326'},
 'totalPatches': 80}
Running predictions...


  0%|          | 0/80 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13...
Writing patch 14...
Writing patch 15...
Writing patch 16...
Writing patch 17...
Writing patch 18...
Writing patch 19...
Writing patch 20...
Writing patch 21...
Writing patch 22...
Writing patch 23...
Writing patch 24...
Writing patch 25...
Writing patch 26...
Writing patch 27...
Writing patch 28...
Writing patch 29...
Writing patch 30...
Writing patch 31...
Writing patch 32...
Writing patch 33...
Writing patch 34...
Writing patch 35...
Writing patch 36...
Writing patch 37...
Writing patch 38...
Writing patch 39...
Writing patch 40...
Writing patch 41...
Writing patch 42...
Writing patch 43...
Writing patch 44...
Writing patch 45...
Writing patch 46...
Writing patch 47...
Writing patch 48...
Writing

In [18]:
doPrediction_multiview_2(ghana_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/m2_global_Cnn_L8SR_S1A_sl/Ghana_with_Multi_2L8SR_S1A_sl.tfrecord.gz']
gs://geebucketwater/m2_global_Cnn_L8SR_S1A_sl/Ghana_with_Multi_2L8SR_S1A_sl.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 2,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -1.9985718441091116,
                                            0.0,
                                            -0.00026949458523585647,
                                            6.081145315847102]},
                'crs': 'EPSG:4326'},
 'totalPatches': 4}
Running predictions...


  0%|          | 0/4 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Started upload task with ID: 5H4ORMD6CID53HARL5CVDBJJ


# Predict Multiview-3

In [ ]:
doPrediction_multiview_3(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

Looking for TFRecord files...
['gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Thai_with_Multi_3L8SR_S1A_sl_el_as300000.tfrecord.gz',
 'gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Thai_with_Multi_3L8SR_S1A_sl_el_as300001.tfrecord.gz',
 'gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Thai_with_Multi_3L8SR_S1A_sl_el_as300002.tfrecord.gz']
gs://geebucketwater/m3_TH_Cnn_L8SR_S1A_sl_el_as3/Thai_with_Multi_3L8SR_S1A_sl_el_as3mixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 6,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            100.30615411937102,
                                            0.0,
                                            -0.00026949458523585647,
                                            18.204359232682105]},
                'crs': 'EPSG:4326'},
 'totalPatches': 42}
Running predictions...


  0%|          | 0/42 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13...
Writing patch 14...
Writing patch 15...
Writing patch 16...
Writing patch 17...
Writing patch 18...
Writing patch 19...
Writing patch 20...
Writing patch 21...
Writing patch 22...
Writing patch 23...
Writing patch 24...
Writing patch 25...
Writing patch 26...
Writing patch 27...
Writing patch 28...
Writing patch 29...
Writing patch 30...
Writing patch 31...
Writing patch 32...
Writing patch 33...
Writing patch 34...
Writing patch 35...
Writing patch 36...
Writing patch 37...
Writing patch 38...
Writing patch 39...
Writing patch 40...
Writing patch 41...
Started upload task with ID: LB6F2YXP3647IGKRKAEFLFMZ


In [ ]:
doPrediction_multiview_3(tb_image_base, user_folder, kernel_buffer, model_custom, "multiview-2_epochs_10_", conf)